In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# upgrade pip, wheel and setuptools
! python -m pip install -U pip wheel setuptools

# install the package
! python -m pip install -U relatio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  

We now need to import the data. On Colab, we can import data we have uploaded on Google Drive. To do that, we first need to associate our Google Drive to this notebook and then specify the path.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Data_course/book_reviews.csv')
df = df.sample(n=100, random_state=12)
df = df.reset_index(drop=True)


In [ ]:
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,review_positive_help,review_tot_help
0,B000FGGOJA,Principles of Quantum Mechanics,19.00,NaN,NaN,21/21,5.0,915494400,Outstanding. Perhaps the best book on the subj...,I own about a dozen or more books on quantum m...,21,21
1,0030706122,The Beet Queen.,NaN,NaN,NaN,11/28,1.0,946339200,Puh-leese!,This book is typical of a lot of the nothingne...,11,28
2,B0006CBNKI,To kill a mockingbird,NaN,A1QDQT0ZAAZHQS,BookMania,26/28,5.0,1038528000,One Of The Best Books I've Ever Read,This book is absolutely brilliant. I've read t...,26,28
3,0393062139,A Mind of its Own: How Your Brain Distorts and...,NaN,A1VX6VPXNQTNUP,Dr. Richard G. Petty,38/40,5.0,1188518400,Can Your Trust Yourself To...,"According to the story, in a survey taken seve...",38,40
4,0307280721,"Eldest (Inheritance, Book 2)",33.97,A3DHQ1D3LIJ4O6,Ms. C. L. Davies,18/26,5.0,1125187200,What's with the negativity???,Ok first things first - on Wednesday I browsed...,18,26


We are interest in using Relatio (https://github.com/relatio-nlp/relatio?tab=readme-ov-file) to do semantic role labeling (this package uses models from AllenNLP library https://github.com/allenai/allennlp).

Here we will try to follow the tutorial when possible.

In [ ]:
from relatio import Preprocessor

p = Preprocessor(
    spacy_model = "en_core_web_md",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    remove_chars = ["\"",'-',"^","(",")","\'","+","&","|","/","{","}",
                    "~","_","`","[","]",">","<","=","*","%","$","@","#","’"],
    stop_words = [],
    n_process = -1,
    batch_size = 100
)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
# Define a regex pattern for the characters you want to remove
regex_pattern = r'["\'\-^()\'\+&|/\{\}~_`\[\]><=*%$@#’]'

# Substitute the specified characters with a space
df['review/text'] =df['review/text'].str.replace(regex_pattern, ' ', regex=True)


df_docs = df[['review/text']].rename(columns={'review/text': 'doc'})


# Add an 'id' column to uniquely identify each row/document
df_docs['id'] = range(1, len(df_docs) + 1)

# Now use the split_into_sentences function
df_revs = p.split_into_sentences(df_docs, output_path=None, progress_bar=True)


Splitting into sentences...


100%|██████████| 100/100 [00:04<00:00, 24.90it/s]


In [ ]:
df_revs.head()

,id,sentence
0,1,I own about a dozen or more books on quantum m...
1,1,He starts out with an excellent chapter on the...
2,1,Shankar is obviously a man with high expositio...
3,1,.I highly recommend this book over Cohen Tanno...
4,1,or Griffiths which deceptively looks like a ...


In [ ]:
! python -m spacy download en_core_web_sm


2024-03-15 11:20:22.701277: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 11:20:22.701360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 11:20:22.703295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 11:20:24.141393: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.7.1
    Uninstalling en-core-web-sm-3.7.1:
      Successfully uninstall

In [ ]:
from relatio import SRL

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = -1
)

srl_res = SRL(df_revs['sentence'], progress_bar=True)

from relatio import extract_roles

roles, sentence_index = extract_roles(
    srl_res,
    used_roles = ["ARG0","B-V","B-ARGM-NEG","B-ARGM-MOD","ARG1","ARG2"],
    only_triplets = True,
    progress_bar = True
)

for d in roles[0:20]: print(d)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Running SRL...


100%|██████████| 120/120 [03:35<00:00,  1.79s/it]


Extracting semantic roles...


100%|██████████| 1199/1199 [00:00<00:00, 7844.15it/s]

{'ARG0': 'I', 'B-V': 'own', 'ARG1': 'about a dozen or more books on quantum mechanics'}
{'ARG0': 'He', 'B-V': 'develops', 'ARG1': 'the theory from the postulates'}
{'ARG0': 'He', 'B-V': 'having', 'ARG1': 'the student in mind'}
{'ARG0': 'I', 'B-V': 'wonder', 'ARG1': 'if he plays the sitar as well as he writes quantum mechanics'}
{'ARG0': 'he', 'B-V': 'plays', 'ARG1': 'the'}
{'ARG0': 'he', 'B-V': 'writes', 'ARG1': 'quantum mechanics'}
{'ARG0': '.I', 'B-V': 'recommend', 'ARG1': 'this book over Cohen Tannoudji too thick'}
{'ARG0': 'Griffiths which', 'B-V': 'looks', 'ARG1': 'like a good book'}
{'ARG0': 'you', 'B-V': 'start', 'ARG1': 'reading the text'}
{'ARG0': 'you', 'B-V': 'reading', 'ARG1': 'the text'}
{'ARG0': 'you', 'B-V': 'attempting', 'ARG1': 'the poor selection of problems'}
{'ARG0': 'I', 'B-V': 'think', 'ARG1': 'the author must have written a bunch of absurd plot twists on bits of paper and pulled them out of a jar as she was writing'}
{'ARG0': 'the author', 'B-V': 'written', 'B-AR

## Semantic Role Labeling (SRL) Roles Explained

In Semantic Role Labeling (SRL), various roles are assigned to words or phrases within a sentence to identify the predicate (usually a verb) and its argument structure. Here's a brief explanation of specific roles:

- **`ARG0`**: Represents the agent or the subject of a sentence in an active voice construction, often the doer of the action.

- **`ARG1`**: Stands for the direct object or the primary argument that the action is directed towards. In sentences with a transitive verb, ARG1 is the entity that receives the action.

- **`ARG2`**: Denotes the indirect object of an action, or additional information about the action, such as the beneficiary, instrument, or sometimes the cause.

- **`B-V`**: Indicates the beginning of the main verb in the sentence, focusing on the action. The `B-` prefix is used to mark the beginning of a phrase structure, in this case, a verb.

- **`B-ARGM-NEG`**: Refers to a negation modifier that marks the beginning of a negation phrase, indicating the presence of negation related to the main verb.

- **`B-ARGM-MOD`**: Represents the beginning of a modal verb or phrase that modifies the main verb, indicating possibility, necessity, or other modalities.

In [ ]:
for d in roles[0:20]:
  try:
    print(d['ARG0'])
  except:
    pass

I
He
He
I
he
he
.I
Griffiths which
you
you
you
I
the author
the author
A mother
A mother
Two orphans
Two the boy
the boy
doesn t


In [ ]:
for d in roles[0:30]:
  try:
    if d['ARG0']=="I":
      print(d)
  except:
    pass


{'ARG0': 'I', 'B-V': 'own', 'ARG1': 'about a dozen or more books on quantum mechanics'}
{'ARG0': 'I', 'B-V': 'wonder', 'ARG1': 'if he plays the sitar as well as he writes quantum mechanics'}
{'ARG0': 'I', 'B-V': 'think', 'ARG1': 'the author must have written a bunch of absurd plot twists on bits of paper and pulled them out of a jar as she was writing'}
{'ARG0': 'I', 'B-V': 'gain', 'ARG1': 'a better appreciation for the storyline , the author , and the moral beliefs that are challenged within these pages'}


In [ ]:
postproc_roles = p.process_roles(roles,
                                 max_length = 50,
                                 progress_bar = True
                                 )

Cleaning phrases for role ARG0...


100%|██████████| 1725/1725 [00:03<00:00, 432.29it/s]


Cleaning phrases for role B-V...


100%|██████████| 1725/1725 [00:05<00:00, 304.78it/s]


Cleaning phrases for role B-ARGM-MOD...


100%|██████████| 186/186 [00:00<00:00, 385.11it/s]


Cleaning phrases for role ARG1...


100%|██████████| 1662/1662 [00:05<00:00, 325.62it/s]


Cleaning phrases for role ARG2...


100%|██████████| 256/256 [00:00<00:00, 289.02it/s]


In [ ]:
for d in postproc_roles[0:50]: print(d)


{'ARG0': 'i', 'B-V': 'own', 'ARG1': 'about a dozen or more book on quantum mechanic'}
{'ARG0': 'he', 'B-V': 'develop', 'ARG1': 'the theory from the postulate'}
{'ARG0': 'he', 'B-V': 'have', 'ARG1': 'the student in mind'}
{'ARG0': 'i', 'B-V': 'wonder'}
{'ARG0': 'he', 'B-V': 'play', 'ARG1': 'the'}
{'ARG0': 'he', 'B-V': 'write', 'ARG1': 'quantum mechanic'}
{'ARG0': '.i', 'B-V': 'recommend', 'ARG1': 'this book over cohen tannoudji too thick'}
{'ARG0': 'griffiths which', 'B-V': 'look', 'ARG1': 'like a good book'}
{'ARG0': 'you', 'B-V': 'start', 'ARG1': 'read the text'}
{'ARG0': 'you', 'B-V': 'read', 'ARG1': 'the text'}
{'ARG0': 'you', 'B-V': 'attempt', 'ARG1': 'the poor selection of problem'}
{'ARG0': 'i', 'B-V': 'think'}
{'ARG0': 'the author', 'B-V': 'write', 'B-ARGM-MOD': 'must', 'ARG1': 'a bunch of absurd plot twist on bit of paper'}
{'ARG0': 'the author', 'B-V': 'pull', 'B-ARGM-MOD': 'must', 'ARG1': 'they', 'ARG2': 'out of a jar'}
{'ARG0': 'a mother', 'B-V': 'abandon', 'ARG1': 'her chil